In [1]:
import json
import pandas as pd


file_path_qa = "/mnt/data/wangshu/hcarag/mintaka/data/mintaka_test.json"
qa_df_raw = pd.read_json(file_path_qa)
qa_df_raw.head(2)


,id,question,translations,questionEntity,answer,category,complexityType
0,fae46b21,What man was a famous American author and also...,{'ar': 'مَن الرجل الذي كان مؤلفًا أمريكيًا مشه...,"[{'name': 'Q1497', 'entityType': 'entity', 'la...","{'answerType': 'entity', 'answer': [{'name': '...",history,intersection
1,bc8713cc,How many Academy Awards has Jake Gyllenhaal be...,{'ar': 'كم عدد جوائز الأوسكار التي ترشح لها ال...,"[{'name': 'Q133313', 'entityType': 'entity', '...","{'answerType': 'numerical', 'answer': [1], 'me...",movies,count


In [3]:
qa_df = qa_df_raw.copy()
qa_df = qa_df[["question", "answer"]]
qa_df.rename(columns={"answer": "answer_raw"}, inplace=True)

qa_df["answers"] = qa_df["answer_raw"].apply(lambda x: x["mention"])

def process_gb_answer(x):
        if isinstance(x, list):
            return "|".join(map(str, x))
        elif isinstance(x, str):
            return x
qa_df["label"] = qa_df["answers"].apply(process_gb_answer)

qa_df.head(2)

,question,answer_raw,answers,label
0,What man was a famous American author and also...,"{'answerType': 'entity', 'answer': [{'name': '...",Mark Twain,Mark Twain
1,How many Academy Awards has Jake Gyllenhaal be...,"{'answerType': 'numerical', 'answer': [1], 'me...",1,1


In [2]:
import sys
import os

# {'answerType': 'entity', 'answer': [{'name': 'Q7245', 'label': {'en': 'Mark Twain', 'ar': 'مارك توين', 'de': 'Mark Twain', 'es': 'Mark Twain', 'fr': 'Mark Twain', 'hi': 'मार्क ट्वैन', 'it': 'Mark Twain', 'ja': 'マーク・トウェイン', 'pt': 'Mark Twain'}}], 'mention': 'Mark Twain'}
# F# 添加 src 文件夹到 sys.path
src_path = "/home/wangshu/rag/hier_graph_rag"
sys.path.append(os.path.abspath(src_path))
from src.utils import relation_embedding

qa_df = qa_df_raw.copy()
qa_df = qa_df[["question", "answer"]]
qa_df.rename(columns={"answer": "answer_raw"}, inplace=True)

qa_df["answers"] = qa_df["answer_raw"].apply(lambda x: x["mention"])


class Args:
    def __init__(self):
        self.embedding_local = False
        self.embedding_api_key = "ollama"
        self.embedding_api_base = "http://localhost:5000/forward"
        self.embedding_model = "nomic-embed-text"


args = Args()
qa_df, _ = relation_embedding(
    qa_df,
    args=args,
    e_colname="question",
    embed_colname="embedding",
    num_workers=48,
    embed_in_ori=True,
)

qa_df.head(2)

local is False
the number of unique question is 4000


Computing embeddings: 100%|██████████| 4000/4000 [00:39<00:00, 102.43it/s]


,question,answer_raw,answers,embedding
0,What man was a famous American author and also...,"{'answerType': 'entity', 'answer': [{'name': '...",Mark Twain,"[-0.022767667, 0.026325747, -0.21478522, -0.03..."
1,How many Academy Awards has Jake Gyllenhaal be...,"{'answerType': 'numerical', 'answer': [1], 'me...",1,"[-0.080311015, 0.028733207, -0.12197757, 0.002..."


In [4]:
save_path_qa = "/mnt/data/wangshu/hcarag/mintaka/QA/mintaka_test_qa.json"
qa_df.to_json(save_path_qa, orient="records", lines=True)

qa_df.head(2)

,question,answer_raw,answers,label
0,What man was a famous American author and also...,"{'answerType': 'entity', 'answer': [{'name': '...",Mark Twain,Mark Twain
1,How many Academy Awards has Jake Gyllenhaal be...,"{'answerType': 'numerical', 'answer': [1], 'me...",1,1
